# Autoencoders

In this notebook we'll try something more advanced - we are going to learn about a special type of a neural network called Autoencoder. Lets first make the imports:

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Conv2D, Flatten, Reshape, Conv2DTranspose, Activation, LSTM
from keras.callbacks import TensorBoard, ModelCheckpoint
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import math
import random

## MNIST Dataset

We will again use our well known MNIST dataset, so lets load it first:

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

We can take a look on how the images look like, e.g. from the testing set. Each image is 28 x 28 pixels size

In [ ]:
x_train.shape

Lets draw some images:

In [ ]:
image_size = 28

image_num = random.randint(0, len(x_train))
img = x_train[image_num]

img = img.reshape(image_size, image_size)

plt.figure()
plt.axis('off')
plt.imshow(img, interpolation='none', cmap='gray')
plt.show()

print("Image label: {}".format(y_train[image_num]))

## Autoencoders

Autoencoders are very interesitng variation of a neural network. In a typical neural network, we have some input data features, based on which we want to predict the labels. So a typical neural network does an aproximation of some function f:

$$
f(x) = y
$$

All of the examples we have shown you before actually use this kind of architecture and do predictions.

Autoencoder is different, with autoencoder you are actually trying to predict the same data as you got on the input:

$$
f(x) = x
$$

Why would you do such a thing? Well let's first look at the autoencoder typical architecture. 

```
               (x)                       (x)
        
               (x)          (x)          (x)
        
input =======> (x) =======> (x) =======> (x) =======> input'
        
               (x)          (x)          (x)
        
               (x)                       (x)
           
             encoder    hidden state   decoder
```

There couple important things here - as explained earlier, during the training the autoencoder is using the same input and output data - it's trying to predict the input from itself. This is done by composing two neural networks - an encoder network and decoder network. Typically the output of the encoder has lower dimensionality than the input itself - it is in some form a compressed representation of the input. But you can also view this as a representation of the most important part of the input, that can be used to reconstruct it back.

You can use different network architectures for both encoder and decoder. For example for image tasks, convolutional layers are common, for sequence based tasks (text processing and similar) recurent layers are typically used.

What you can use autoencoder for some very specific tasks, for example:
- create a compressed representation of the input, that can be used to reconstruct it (though this is a very bad compression algorithm, as it can compress only very specific data)
- because the autoencoder learns the most important part of the input, it can be actually used for removing the noise from the data - e.g. denoising the images
- autoencoders can be used for detecting anomalies in the data, such as frauds in financial transactions, or iot sensor malfunction. This is because they can learn how "normal" data look like, and if the predicted output differs from the input, we can consider it as anomaly.

As an example for autoencoder we will show how to denoise images from MNIST.

We will use already loaded data, we just need to reshape it back to 28 x 28 pixels, as we are going to use convolutional layers this time

In [ ]:
x_train = np.reshape(x_train, [-1, image_size, image_size, 1]) / 255
x_test = np.reshape(x_test, [-1, image_size, image_size, 1]) / 255

In [ ]:
x_train.shape

Now we are going to add some random noise to the images

In [ ]:
noise = np.random.normal(loc=0.5, scale=0.5, size=x_train.shape)
x_train_noisy = x_train + noise
noise = np.random.normal(loc=0.5, scale=0.5, size=x_test.shape)
x_test_noisy = x_test + noise

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

Now we have a transformed dataset, lets draw some samples:

In [ ]:
image_index = random.randint(0, len(x_train_noisy))
img = x_train_noisy[image_index]
img.shape
img_plt = img.reshape(image_size, image_size
                    )
img_plt = (img_plt * 255).astype(np.uint8)

plt.imshow(img_plt, interpolation='none', cmap='gray')
plt.show()

print("Image labels: {}".format(y_train[image_index]))

We are going to create an encoder with convolutional layers that are going to create a hidden representation of the image.

In [ ]:
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
latent_dim = 16

Note here the convolutional layers that are transforming the image to smaller and smaller sizes - you will need to code the oposite transformation in the decoder.

In [ ]:
# Encoder
inputs = Input(shape=input_shape, name='encoder_input')
x = inputs

x = Conv2D(filters=32,
               kernel_size=kernel_size,
               strides=2,
               activation='relu',
               padding='same')(x)
x = Conv2D(filters=64,
               kernel_size=kernel_size,
               strides=2,
               activation='relu',
               padding='same')(x)

In [ ]:
# Shape info needed to build Decoder Model
shape = K.int_shape(x)

# Generate the latent vector
x = Flatten()(x)
latent = Dense(latent_dim, name='latent_vector')(x)

# Instantiate Encoder Model
encoder = Model(inputs, latent, name='encoder')
encoder.summary()

As we see, the output of the encoder is just 16 values - much less than original 784. Now lets create a decoder from this

In [ ]:
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

Your turn now, define the decoder convolutional layers - they have similar parameters as the two layers above, just you need to use Conv2DTranspose layer instead of Conv2D and reverse the filter numbers:

In [ ]:
# TODO define two decoder convolutional layers, they should look exactly the same as in encoder, just swapped

In [ ]:
x = Conv2DTranspose(filters=1,
                    kernel_size=kernel_size,
                    padding='same')(x)

outputs = Activation('sigmoid', name='decoder_output')(x)

# Instantiate Decoder Model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()


Train the model (we can also show a summary)

In [ ]:
autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')
autoencoder.summary()

autoencoder.compile(loss='mse', optimizer='adam')

# Train the autoencoder
autoencoder.fit(x_train_noisy,
                x_train,
                validation_data=(x_test_noisy, x_test),
                epochs=2,
                batch_size=batch_size)

And make predictions on the whole test set

In [ ]:
x_decoded = autoencoder.predict(x_test_noisy)

We can now show the picture after denoising

In [ ]:
index = random.randint(0, len(x_test_noisy))
img = x_test_noisy[index]
img_plt = img.reshape(image_size, image_size)
img_plt = (img_plt * 255).astype(np.uint8)

plt.imshow(img_plt, interpolation='none', cmap='gray')
plt.show()

img = x_decoded[index]
img_plt = img.reshape(image_size, image_size)
img_plt = (img_plt * 255).astype(np.uint8)

plt.imshow(img_plt, interpolation='none', cmap='gray')
plt.show()